In [5]:
!pip install -qU langchain-groq

In [6]:
#from dotenv import load_dotenv
#load_dotenv() # take environment variables from .env.

from google.colab import userdata
import os

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

In [9]:
#from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize the Groq model
model = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_retries=2,
)

prompt = PromptTemplate(
        template="Explain about {topic} in 100 words.",
        input_variables=["topic"]
    )

chain_without_parser = prompt | model
result_without = chain_without_parser.invoke({"topic": "Large Language Models"})
print(result_without)

content='Large Language Models (LLMs) are artificial intelligence (AI) systems that can process and generate human-like language. They are trained on vast amounts of text data, allowing them to learn patterns, relationships, and context. LLMs use neural networks to analyze and generate text, making them capable of tasks such as language translation, text summarization, and even creative writing. They can also understand nuances like idioms, sarcasm, and figurative language. LLMs have revolutionized natural language processing (NLP) and have numerous applications in areas like customer service, content generation, and research. Their capabilities continue to improve with advancements in AI technology.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 133, 'prompt_tokens': 46, 'total_tokens': 179, 'completion_time': 0.165395533, 'completion_tokens_details': None, 'prompt_time': 0.005014771, 'prompt_tokens_details': None, 'queue_time': 0.075893339, 'total_time'

In [10]:
parser = StrOutputParser()
chain_with_parser = prompt | model | parser
result_with = chain_with_parser.invoke({"topic": "Large Language Models"})
print(result_with)

Large Language Models (LLMs) are artificial intelligence (AI) systems that can process and generate human-like language. They are trained on vast amounts of text data, allowing them to learn patterns, relationships, and context. LLMs use neural networks to analyze and generate text, making them capable of tasks such as language translation, text summarization, and even creative writing. They can also understand nuances like idioms, sarcasm, and figurative language. LLMs have revolutionized natural language processing (NLP) and have numerous applications in areas like customer service, content generation, and research. Their capabilities continue to improve with advancements in AI technology.


In [11]:
prompt_without_parser = PromptTemplate(
        template="Analyze the movie {movie_title} and provide details about genre, rating, and key themes.",
        input_variables=["movie_title"]
    )

In [12]:
chain_without_parser = prompt_without_parser | model
result_without = chain_without_parser.invoke({"movie_title": "The Matrix"})
print(result_without)

content='**Movie Analysis: The Matrix (1999)**\n\n**Genre:** Science Fiction, Action, Thriller\n\n**Rating:** The Matrix (1999) received a PG-13 rating from the Motion Picture Association of America (MPAA) for sequences of sci-fi violence and action.\n\n**Plot Summary:**\n\nThe Matrix is a thought-provoking science fiction film written and directed by the Wachowskis. The movie takes place in a dystopian future where humanity is unknowingly trapped within a simulated reality called the Matrix. Created by intelligent machines, the Matrix is a complex web of virtual reality that keeps humans docile and ignorant of their enslavement. The story follows Neo (played by Keanu Reeves), a computer hacker who is eventually awakened to the true nature of the world. With the help of the resistance leader Morpheus (played by Laurence Fishburne), Neo embarks on a perilous journey to understand the Matrix and free humanity from its grasp.\n\n**Key Themes:**\n\n1. **Reality and Perception:** The Matrix

In [13]:
from langchain_core.output_parsers import JsonOutputParser

In [18]:
parser = JsonOutputParser()
format_instructions = parser.get_format_instructions()


prompt_with_parser = PromptTemplate(
    template="""Analyze the movie {movie_title} and provide details about genre, rating, and key themes.
    Ensure all numerical values in the JSON, especially percentages, are strictly numerical or enclosed in quotes. {format_instructions}""",
    input_variables=["movie_title"],
    partial_variables={"format_instructions": format_instructions}
    )

chain_with_parser = prompt_with_parser | model | parser
result_with = chain_with_parser.invoke({"movie_title": "The Matrix"})
print(result_with)

{'title': 'The Matrix', 'genre': ['Science Fiction', 'Action', 'Thriller'], 'rating': {'IMDB': '8.7', 'Rotten Tomatoes': '87%', 'Metacritic': '73'}, 'key_themes': ['Reality vs. Illusion', 'Free Will vs. Determinism', 'Revolution and Social Change', 'The Nature of Consciousness', 'The Impact of Technology on Society'], 'plot_summary': 'In a dystopian future, humanity is unknowingly trapped within a simulated reality called the Matrix. A computer hacker named Neo is eventually awakened to the true nature of the world and joins a group of rebels to free humanity from its shackles.', 'release_year': 1999, 'runtime': '136 minutes'}


In [15]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

In [16]:
# Define Pydantic model
class Person(BaseModel):
    name: str = Field(description="Person's full name")
    age: int = Field(description="Person's age in years", ge=0, le=150)
    occupation: str = Field(description="Person's job or profession")
    skills: List[str] = Field(description="List of person's key skills")

In [17]:
prompt_without_parser = PromptTemplate(
    template="Create a fictional person profile for a {profession}.",
    input_variables=["profession"]
)

chain_without_parser = prompt_without_parser | model
result_without = chain_without_parser.invoke({"profession": "data scientist"})

print("BEFORE PARSER:")
print("Prompt used:", prompt_without_parser.invoke({"profession": "data scientist"}))
print(f"Type: {type(result_without)}")
print(f"result_without: {result_without}")
print()

# WITH PARSER - Modified prompt with format instructions
parser = PydanticOutputParser(pydantic_object=Person)
format_instructions = parser.get_format_instructions()


prompt_with_parser = PromptTemplate(
    template="Create a fictional person profile for a {profession}.\n{format_instructions}",
    input_variables=["profession"],
    partial_variables={"format_instructions": format_instructions}
)

chain_with_parser = prompt_with_parser | model | parser
result_with = chain_with_parser.invoke({"profession": "data scientist"})

print("AFTER PARSER:")
print("Prompt used:", prompt_with_parser.invoke({"profession": "data scientist"}))
print(f"Type: {type(result_with)}")
print(f"result_with: {result_with}")

BEFORE PARSER:
Prompt used: text='Create a fictional person profile for a data scientist.'
Type: <class 'langchain_core.messages.ai.AIMessage'>
result_without: content="**Name:** Dr. Maya Jensen\n\n**Age:** 32\n\n**Nationality:** Danish-American\n\n**Education:**\n\n- **Bachelor's Degree:** Bachelor of Science in Mathematics, University of Copenhagen (2012-2015)\n- **Master's Degree:** Master of Science in Data Science, University of California, Berkeley (2015-2017)\n- **Ph.D.:** Ph.D. in Machine Learning, Stanford University (2017-2022)\n\n**Work Experience:**\n\n- **Data Scientist, Google (2022-Present):** Currently working as a data scientist at Google, focusing on developing predictive models for Google Ads and Google Analytics.\n- **Research Scientist, Stanford University (2020-2022):** Conducted research on deep learning and natural language processing as a research scientist at Stanford University.\n- **Data Analyst, McKinsey & Company (2017-2020):** Worked as a data analyst at 